In [0]:
from google.colab import files
uploaded = files.upload()

In [165]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c competitive-data-science-predict-future-sales

mkdir: cannot create directory ‘/root/.kaggle’: File exists
item_categories.csv: Skipping, found more recently modified local copy (use --force to force download)
items.csv: Skipping, found more recently modified local copy (use --force to force download)
shops.csv: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.gz: Skipping, found more recently modified local copy (use --force to force download)
test.csv.gz: Skipping, found more recently modified local copy (use --force to force download)
sales_train.csv.gz: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import seaborn as sns

train = pd.read_csv('/content/sales_train.csv.gz', header=0, sep=',', quotechar='"')
test = pd.read_csv('/content/test.csv.gz', header=0, sep=',', quotechar='"',index_col="ID")
subm = pd.read_csv('/content/sample_submission.csv.gz', header=0, sep=',', quotechar='"')
item = pd.read_csv('/content/items.csv', header=0, sep=',', quotechar='"')
cate = pd.read_csv('/content/item_categories.csv', header=0, sep=',', quotechar='"')
shop = pd.read_csv('/content/shops.csv', header=0, sep=',', quotechar='"')

In [0]:
train = train.join(item[['item_id','item_category_id']].set_index('item_id'),on='item_id')
train = train.drop(['date','date_block_num'],axis = 1)

In [0]:
test = test.join(item[['item_id','item_category_id']].set_index('item_id'),on='item_id')

In [0]:
price = train.drop_duplicates('item_id')[['item_id','item_price']]

In [0]:
test = test.join(price[['item_id','item_price']].set_index('item_id'),on='item_id')
test = test.fillna(test.item_price.mean())

In [0]:
trainlabel = train['item_cnt_day']
traindata = train.drop('item_cnt_day',axis = 1)

In [0]:
from tensorflow import keras

def build_model():
  model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=[len(traindata.keys())]),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
  ])

  optimizer = keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [0]:
model = build_model()

In [0]:
model.fit(traindata,trainlabel,epochs=10,batch_size=50000)

In [0]:
predict = model.predict(test)

In [0]:
from sklearn.metrics import r2_score
r2_score(subm['item_cnt_month'].values,predict)

In [0]:
from sklearn.ensemble import RandomForestRegressor

RFR = RandomForestRegressor()

In [146]:
RFR.fit(traindata,trainlabel)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [0]:
RFRpredict = RFR.predict(test)

In [152]:
r2_score(subm['item_cnt_month'],RFRpredict)

0.0

In [155]:
RFRpredict

array([1.27591424, 1.34062012, 1.27591424, ..., 1.4952381 , 1.40803907,
       1.78261221])

In [0]:
output  = pd.DataFrame({'ID':subm['ID'],'item_cnt_month':RFRpredict})

In [184]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
output.to_csv('Submission.csv',index='ID')
!cp Submission.csv drive/My\ Drive/

In [186]:
!kaggle competitions submit -c competitive-data-science-predict-future-sales -f Submission.csv -m "Message"

100% 6.32M/6.32M [00:08<00:00, 736kB/s]
Successfully submitted to Predict Future Sales

In [179]:
output

,ID,item_cnt_month
0,0,1.275914
1,1,1.340620
2,2,1.275914
3,3,1.340620
4,4,1.340620
...,...,...
214195,214195,1.495238
214196,214196,1.200000
214197,214197,1.495238
214198,214198,1.408039
